# All types vs Just Contracts


## Setup

In [1]:
1+1

2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from pathlib import Path
import glob

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

Libraries imported successfully!


## Load Most Recent Datasets

In [9]:
!ls ../data/round2

transactions_normalized_2026-01-09_09-58-16-5000-alltypes.json


In [10]:
transactions_file = Path('../data/round2/transactions_normalized_2026-01-09_09-58-16-5000-alltypes.json')
with open(transactions_file, 'r') as f:
    transactions_data = json.load(f)
transactions_df = pd.DataFrame(transactions_data)

In [13]:
transactions_df.shape

(5000, 22)

In [11]:
transactions_df.head()

,transaction_id,award_id,action_date,action_type,action_type_description,modification_number,federal_action_obligation,total_dollars_obligated,award_type,award_description,period_of_performance_start_date,period_of_performance_current_end_date,awarding_agency_name,awarding_sub_agency_name,funding_agency_name,recipient_name,recipient_uei,naics_code,product_or_service_code,place_of_performance_state,ingested_at,source_url
0,278754767,19AQMM25F0157,2025-02-28,C,REVISION,P00002,32438.56,32438.56,DELIVERY ORDER,INCREMENTAL FUNDING ACTION ONLY.,2025-01-01,None,Department of State,Department of State,Department of State,"KYMBER CONSULTING GROUP, LLC",FBBNK1NLMXZ9,541611,R408,DC,2026-01-09T14:58:16.519Z,https://www.usaspending.gov/award/19AQMM25F0157
1,269654839,P063P240233,2025-02-28,C,REVISION,164,11423.00,11423.00,"DIRECT PAYMENT FOR SPECIFIED USE, AS A SUBSIDY...",GRANT PROGRAM,None,None,Department of Education,Department of Education,Department of Education,MICHIGAN STATE UNIVERSITY,R28EKN92ZTZ9,None,None,MI,2026-01-09T14:58:16.519Z,https://www.usaspending.gov/award/P063P240233
2,268300385,LI-143035888-439061,2025-02-28,C,REVISION,51,209405.00,209405.00,"DIRECT PAYMENT FOR SPECIFIED USE, AS A SUBSIDY...",THE LIFELINE PROGRAM HELPS MAKE COMMUNICATIONS...,None,None,Federal Communications Commission,Federal Communications Commission,Federal Communications Commission,AMERIMEX COMMUNICATIONS CORP,D3VAYKS53TP7,None,None,OK,2026-01-09T14:58:16.519Z,https://www.usaspending.gov/award/LI-143035888...
3,306509621,HC108423F0159,2025-02-28,M,M,P00005,11028.00,11028.00,BPA CALL,MICROSOFT SUBSCRIPTION AND SA,2023-07-01,None,Department of Defense,Defense Information Systems Agency,Department of Defense,DELL MARKETING L.P.,HJLSY7RZV8X6,511210,7A21,VA,2026-01-09T14:58:16.519Z,https://www.usaspending.gov/award/HC108423F0159
4,269656505,P063P242371,2025-02-28,C,REVISION,38,3697.00,3697.00,"DIRECT PAYMENT FOR SPECIFIED USE, AS A SUBSIDY...",GRANT PROGRAM,None,None,Department of Education,Department of Education,Department of Education,EASTERN MENNONITE UNIVERSITY,ZKGYWDLKCU93,None,None,VA,2026-01-09T14:58:16.519Z,https://www.usaspending.gov/award/P063P242371


In [12]:
good_types = ['DELIVERY ORDER', 'DEFINITIVE CONTRACT', 'BPA CALL', 'PURCHASE ORDER']

In [16]:
# Filter transactions into Contracts vs Non-Contracts
contracts_df = transactions_df[transactions_df['award_type'].isin(good_types)]
non_contracts_df = transactions_df[~transactions_df['award_type'].isin(good_types)]

print(f"Total transactions: {len(transactions_df):,}")
print(f"Contracts: {len(contracts_df):,} ({len(contracts_df)/len(transactions_df)*100:.1f}%)")
print(f"Non-Contracts: {len(non_contracts_df):,} ({len(non_contracts_df)/len(transactions_df)*100:.1f}%)")

Total transactions: 5,000
Contracts: 984 (19.7%)
Non-Contracts: 4,016 (80.3%)


In [17]:
# Aggregation: Count and Sum by Contract Type
summary_data = {
    'Category': ['Contracts', 'Non-Contracts', 'Total'],
    'Count': [
        len(contracts_df),
        len(non_contracts_df),
        len(transactions_df)
    ],
    'Total Obligation': [
        contracts_df['federal_action_obligation'].sum(),
        non_contracts_df['federal_action_obligation'].sum(),
        transactions_df['federal_action_obligation'].sum()
    ],
    'Avg Obligation': [
        contracts_df['federal_action_obligation'].mean(),
        non_contracts_df['federal_action_obligation'].mean(),
        transactions_df['federal_action_obligation'].mean()
    ]
}

summary_df = pd.DataFrame(summary_data)
summary_df['% of Count'] = (summary_df['Count'] / len(transactions_df) * 100).round(1)
summary_df['% of Value'] = (summary_df['Total Obligation'] / transactions_df['federal_action_obligation'].sum() * 100).round(1)

print("=== CONTRACTS vs NON-CONTRACTS SUMMARY ===")
print(summary_df.to_string(index=False))

# Show breakdown of non-contract award types
print("\n=== NON-CONTRACT AWARD TYPES BREAKDOWN ===")
non_contract_breakdown = non_contracts_df.groupby('award_type').agg({
    'transaction_id': 'count',
    'federal_action_obligation': 'sum'
}).sort_values('federal_action_obligation', ascending=False)
non_contract_breakdown.columns = ['Count', 'Total Obligation']
print(non_contract_breakdown.to_string())

=== CONTRACTS vs NON-CONTRACTS SUMMARY ===
     Category  Count  Total Obligation  Avg Obligation  % of Count  % of Value
    Contracts    984     1739735825.77      1768024.21       19.70        8.60
Non-Contracts   4016    18585579320.60      4627883.30       80.30       91.40
        Total   5000    20325315146.37      4065063.03      100.00      100.00

=== NON-CONTRACT AWARD TYPES BREAKDOWN ===
                                                                                                   Count  Total Obligation
award_type                                                                                                                
DIRECT PAYMENT WITH UNRESTRICTED USE (RETIREMENT, PENSION, VETERANS BENEFITS, ETC.) (D)             1527    17870983417.00
FORMULA GRANT (A)                                                                                    245      341571376.68
DIRECT PAYMENT FOR SPECIFIED USE, AS A SUBSIDY OR OTHER NON-REIMBURSABLE DIRECT FINANCIAL AID (C)   1546 

In [20]:
transactions_df.action_date.max()

'2025-02-28'

In [21]:
1_900_000 / 365

5205.479452054795